<h1> Emoji DB for James </h1>

In [1]:
from __future__ import division
import json
import pandas as pd
import re, collections
import numpy as np
import os
base_dir=os.path.expanduser('~')
import datetime
import time
import psycopg2

<h1> Connect to the emoji_db </h1>

In [1]:
#connect
conn = psycopg2.connect("host=52.40.90.162 port=5432 dbname=emoji_db user=WizzKid password=IVapeBro")
cur = conn.cursor()

NameError: name 'psycopg2' is not defined

In [5]:
cur.execute("SELECT mapjson from emoji_map limit 5;")
result=cur.fetchall()

ProgrammingError: permission denied for relation emoji_map


<h1> search by date time on hashtags </h1>

In [ ]:
start_time=datetime.datetime.now()-datetime.timedelta(hours=1)
word="%#%"
cur.execute("SELECT text from tweet_dump WHERE (created_at > %s and lang='en' and text LIKE %s ) \
order by id DESC limit 10;",(start_time,word))
results=cur.fetchall()

<h1> tweet_dump table </h1>

In [ ]:
cur.execute(\
"CREATE TABLE tweet_dump (id serial PRIMARY KEY,\
    date timestamp,\
    created_at timestamp,\
    text text,\
    retweet_count integer,\
    favorite_count integer,\
    lang varchar(5),\
    geo json,\
    coordinates json,\
    time_zone varchar(40),\
    name varchar(40),\
    user_name varchar(40)\
    );")
conn.commit()

<h1> has_emoji table tweet_id= id of tweet_dump table) </h1>

In [ ]:
#cur.execute(\
"CREATE TABLE has_emoji (id serial PRIMARY KEY,\
    tweet_id integer,\
    has_emoji bool\
    );")
#conn.commit() #submit change to db

<h1> emoji_tweet table (tweet_id= id of tweet_dump table) </h1>

In [ ]:
#cur.execute\
("CREATE TABLE emoji_tweet (id serial PRIMARY KEY,\
    tweet_id integer,\
    date timestamp,\
    created_at timestamp,\
    text text,\
    retweet_count integer,\
    favorite_count integer,\
    lang varchar(5),\
    geo json,\
    coordinates json,\
    time_zone varchar(40),\
    name varchar(40),\
    user_name varchar(40),\
    emojiLabel text[],\
    emojiLabelFaceFilter bool[],\
    emojiCount integer[],\
    emojiCountSum integer,\
    emojiTypes integer,\
    prev_word text[],\
    next_word text[],\
    prev_sentence text[],\
    next_sentence text[],\
    mostFreqWord text,\
    mostFreqWordCount integer,\
    newlineCount integer,\
    emojiSkinLabel text[],\
    emojiSkinCount integer[],\
    emojiSkinCountSum integer,\
    emojiSkinTypes integer,\
    emojistrLabel text[],\
    emojistrCount integer[],\
    emojistrLen integer[],\
    emojistrTypes integer,\
    emojistr_prev_word text[],\
    emojistr_next_word text[],\
    emojistr_prev_sentence text[],\
    emojistr_next_sentence text[],\
    emojiPatternLabel text[],\
    emojiPatternCount integer[],\
    emojiPatternLen integer[],\
    emojiPatternTypes integer\
    );")
#cur.commit()

<h1> Example Searches </h1>

<h3> look for regex in the tweet </h3>

In [9]:
cur.execute("SELECT text from emoji_tweet WHERE text LIKE '%🍆%' limit 5;")
result=cur.fetchall()
for text in result:
    print(text[0])
#or build a list# result = [text[0] for text in cur.fetchall()]

RT @pabebegirl16161: #메이저놀이터
#메이저놀이터

사-설-토-토-사-이-트-추-천

🍆🍆 https://t.co/7IQSPOiCnq 🍆🍆 h
✅
✅
✅
✅
✅
✅
✅
✈사다리연승이벤트✈
⚠⚠ 신규첫충10%+매일첫충10% ⚠⚠
@a_o_ru
謝謝那須 🍆
Tuesday means one thing #Geordieshore 🍆🍆🍆🍆
Online 24/7 when are u getting laid 🍆🍆💦💦💦
RT @KingJohnnyMackG: 👑 #JMG 👑 💦 #TittyTuesday 💦
🍒 #SweetCherryPops 🍒🍆 #BBCKings 🍆
💪 #MrFit33 💪🏆 #TexasFinest 🏆 get 👇 with the #Squad https://t.…


<h3> select on tweets with geo info (json object)...by defualt twitter users have this set off, rare ~1/20 </h3>

In [ ]:
#example:
#{"full_name": "Lomita, CA", "country": "United States", "place_type": "city", "coordinates": [[[-118.328804, 33.774937], [-118.328804, 33.807678], [-118.306568, 33.807678], [-118.306568, 33.774937]]], "country_code": "US", "id": "f2f180b090191151", "name": "Lomita"}
cur.execute("select geo from tweet_dump where geo->'full_name' IS NOT NULL order by id DESC limit 5;")
result=cur.fetchall()

<h3> select time_zone info ... most tweets for have time_zone </h3>

In [ ]:
cur.execute("SELECT time_zone from tweet_dump WHERE text LIKE '%🍆%' limit 5;")
result=cur.fetchall()

<h3> join table example. Find last entry analyzed by workers in has emoji and search rest of tweet_dump </h3>

In [ ]:
cur.execute("SELECT date from tweet_dump a join has_emoji b on a.id=b.tweet_id WHERE (a.date > %s and a.text != '' and b.has_emoji=True);",last_date)
#group by hour? SUM the case has_emoji=True and has_emoji=False
#in pandas #EmjFraction = (1+1/df.resample("H",how=['mean']))**-1 #calcuate emoji_usage fraction .... has_emoji/all_tweets

In [ ]:
#core number is either 0 or 1
cur.execute("SELECT tweet_id from has_emoji WHERE MOD(tweet_id,2)=%s order by tweet_id DESC limit 1;",(core_number,))#find last processed id
last_id=cur.fetchone()
cur.execute("SELECT * from tweet_dump WHERE (id>%s AND MOD(id,2)=%s) LIMIT 10000;",(last_id,core_number)) a=cur.fetchall()
len(a)

<h3> select on art </h3>

In [ ]:
cur.execute("SELECT text from emoji_tweet WHERE (emojiCountSum > 30) order by random() limit 40;")
result=[text[0] for text in cur.fetchall()]

<h1> Search for skin tone info </h1>

In [ ]:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiSkinLabel) as Label, unnest(emojiSkinCount) as Freq FROM emoji_tweet WHERE (emojiSkinCountSum>0 AND LOWER(text) LIKE '%beach%')) as T group by T.Label order by TFreq DESC;"

<h1> Used on website </h1>

In [ ]:
#most freq emoji, count all:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiLabel) as Label, unnest(emojiCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label order by TFreq DESC limit 15;"
#with the face filter:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiLabel) as Label, unnest(emojiCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T WHERE(T.FF is True) group by T.Label order by TFreq DESC limit 15;"

#select emoji str
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label order by TFreq DESC limit 15;"
#with FaceFilter on emojiLabel:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T WHERE(FF is True) group by T.Label order by TFreq DESC limit 15;"
#cut all tweets with a face: #the cut happens after groupy so it's not quite accurate
"SELECT T.Label,SUM(T.Freq) as TFreq,bool_and(T.FF) as prodFF From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label  HAVING(bool_and(FF) is True) order by TFreq DESC limit 15;"
#cut all tweets with face... should be faster, using: NOT (emojiLabelFaceFilter @> ARRAY[False])
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq FROM emoji_tweet WHERE (emojistrTypes>0 AND LOWER(text) LIKE '% %' and NOT(emojiLabelFaceFilter @> ARRAY[False]))) as T group by T.Label order by TFreq DESC limit 15;"


#select emojiPattern
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiPatternLabel) as Label, unnest(emojiPatternCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label order by TFreq DESC limit 15;"

#Select ALL
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(array_cat(emojiLabel,array_cat(emojistrLabel,emojiPatternLabel))) as Label, unnest(array_cat(emojiCount,array_cat(emojistrCount,emojiPatternCount))) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '%dog%')) as T group by T.Label order by TFreq DESC limit 15;"

<h1> Group by time_zone </h1>

In [ ]:
SELECT Q.zone, Q.maximum, label FROM ((SELECT zone, MAX(tfreq) as maximum from (SELECT T.zone, T.Label,SUM(T.Freq) as TFreq From (SELECT time_zone as zone, unnest(emojiLabel) as Label, unnest(emojiCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.zone,T.Label) as F group by zone order by maximum DESC limit 15) as Q join (SELECT T.zone, T.Label,SUM(T.Freq) as TFreq From (SELECT time_zone as zone, unnest(emojiLabel) as Label, unnest(emojiCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.zone,T.Label) as R on Q.maximum=R.tfreq) order by Q.maximum DESC;